<a href="https://colab.research.google.com/github/elisasmenendez/cajoo-learning/blob/master/datasets/2_census/census.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import keras

# Pre-processing

In [ ]:
base = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/census.csv')
attributes = base.iloc[:,0:14].values
classes = base.iloc[:,14].values
# base

In [ ]:
# Transform nominal in discrete - no dummies here 
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
attributes[:,1] = labelencoder.fit_transform(attributes[:,1])
attributes[:,3] = labelencoder.fit_transform(attributes[:,3])
attributes[:,5] = labelencoder.fit_transform(attributes[:,5])
attributes[:,6] = labelencoder.fit_transform(attributes[:,6])
attributes[:,7] = labelencoder.fit_transform(attributes[:,7])
attributes[:,8] = labelencoder.fit_transform(attributes[:,8])
attributes[:,9] = labelencoder.fit_transform(attributes[:,9])
attributes[:,13] = labelencoder.fit_transform(attributes[:,13])

In [ ]:
# Transform nominal in dummies
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [1,3,5,6,7,8,9,13])],remainder='passthrough')
attributes = onehotencorder.fit_transform(attributes).toarray()

In [ ]:
# Transform class in discrete
from sklearn.preprocessing import LabelEncoder
labelencorder_classe = LabelEncoder()
classes = labelencorder_classe.fit_transform(classes)

In [ ]:
# Scale all attributes
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
attributes = scaler.fit_transform(attributes)

In [ ]:
# Split in training and test
from sklearn.model_selection import train_test_split
att_training, att_test, cls_training, cls_test = train_test_split(attributes, classes, test_size=0.15, random_state=0)

# Base line - 0.755

In [ ]:
import collections
collections.Counter(cls_test)

3693/(3693 + 1192)

# Naive Bayes - 0.805

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(att_training, cls_training)
results = classifier.predict(att_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
accuracy = accuracy_score(cls_test, results)
matrix = confusion_matrix(cls_test, results)

In [ ]:
# Dummies + scaling - 0.47
# Dummine + no scaling - 0.79
# No dummies + scaling - 0.805
# Dummies + scaling numerical - ?
accuracy 

0.8057318321392016

In [ ]:
matrix

# Decision Trees - 0.810

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion='entropy')
classifier.fit(att_training, cls_training)
results = classifier.predict(att_test)

from sklearn.metrics import confusion_matrix, accuracy_score
accuracy = accuracy_score(cls_test, results)
matrix = confusion_matrix(cls_test, results)

In [ ]:
# Dummies + scaling - 0.809
# Dummies + no scaling - 0.810
# No dummies + scaling - 0.810
accuracy

0.8104401228249745

# Random Forest - 0.852

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)
classifier.fit(att_training, cls_training)
results = classifier.predict(att_test)

from sklearn.metrics import confusion_matrix, accuracy_score
accuracy = accuracy_score(cls_test, results)
matrix = confusion_matrix(cls_test, results)

In [ ]:
# No dummies + scaling + 10 estimators - 0.841
# No dummies + scaling + 50 estimators - 0.849
# No dummies + scaling + 100 estimators - 0.852
accuracy

# kNN - 0.821

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p = 2) # euclidian distance
classifier.fit(att_training,cls_training)
results = classifier.predict(att_test)

from sklearn.metrics import confusion_matrix, accuracy_score
accuracy = accuracy_score(cls_test, results)
matrix = confusion_matrix(cls_test, results)

In [ ]:
# No dummies + scaling + 5 neighbors - 0.841
accuracy

0.8219037871033776

# Logistic Regression - 0.849

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 1, solver='lbfgs')
classifier.fit(att_training,cls_training)
results = classifier.predict(att_test)

from sklearn.metrics import confusion_matrix, accuracy_score
accuracy = accuracy_score(cls_test, results)
matrix = confusion_matrix(cls_test, results)

In [ ]:
# Dummies + scaling - 0.849
# Dummine + no scaling - 0.794
# No dummies + scaling - 0.818
accuracy

0.8184237461617195

In [ ]:
print(classifier.intercept_) # global coeficient
print(classifier.coef_) # attributes coeficient

In [ ]:
results2 = classifier.predict_proba(att_test) # probabilities and complements
print(results2)

# SVM - 0.851

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 1, C = 2.0)
classifier.fit(att_training,cls_training)
results = classifier.predict(att_test)

from sklearn.metrics import confusion_matrix, accuracy_score
accuracy = accuracy_score(cls_test, results)
matrix = confusion_matrix(cls_test, results)

In [ ]:
# pre = no dummies + scaling
#   linear  kernel & c = 1.0 - 0.813
#   poly    kernel & c = 1.0 - 0.839
#   sigmoid kernel & c = 1.0 - 0.749
#   rbf     kernel & c = 1.0 - 0.845
#   rbf     kernel & c = 2.0 - 0.845

# pre = dummies + scaling
#   linear  kernel & c = 1.0 - 0.851
#   linear  kernel & c = 2.0 - 0.851
#   rbf     kernel & c = 2.0 - 0.848
accuracy

0.8507676560900717

# Neural Networks - sklearn - 0.8405

In [ ]:
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier(verbose = True, max_iter=1000, tol=0.000010)
classifier.fit(att_training,cls_training)
results = classifier.predict(att_test)

from sklearn.metrics import confusion_matrix, accuracy_score
accuracy = accuracy_score(cls_test, results)
matrix = confusion_matrix(cls_test, results)

In [ ]:
# pre = no dummies + scaling
#   iter = 1000 & tol = 0.000010 - 0.840
accuracy

# Neural Networks - keras - 0.845

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
classifier = Sequential()
classifier.add(Dense(units = 8, activation = 'relu', input_dim = 14))
classifier.add(Dense(units = 8, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(att_training, cls_training, batch_size = 10, nb_epoch = 100)
results = classifier.predict(att_test)
results = (results > 0.5)

from sklearn.metrics import confusion_matrix, accuracy_score
accuracy = accuracy_score(cls_test, results)
matrix = confusion_matrix(cls_test, results)

In [ ]:
# no dummies + scaling - 0.845
# dummies + scaling - 
accuracy

0.84503582395087